<a href="https://colab.research.google.com/github/sanket-ghodake/handwritten-character-recognition/blob/main/colab_notebooks/input_pipeline_for_dataset_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

remove directory completely

In [ ]:
%rm -rf '/content/dataset'

unzip file

In [ ]:
!unzip "/content/dataset.zip"

import

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
! pip install ipyplot
import ipyplot
import glob
import os
import urllib.request
import zipfile
import matplotlib.pyplot as plt
from tqdm import tqdm

functions

In [ ]:
def crop_image_remove_white(gray, pixel_value=255):
    # gray = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    
    # STEP 1
    # OTSU threshold lines are added by me. 
    (thresh, im_bw) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    gray = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)[1]

    crop_rows = gray[~np.all(gray == pixel_value, axis=1), :]
    cropped_image = crop_rows[:, ~np.all(crop_rows == pixel_value, axis=0)]
    # cv2_imshow(cropped_image)
    # print(cropped_image.shape)
    return cropped_image

# STEP 2 (INPUT FIRST STEP)

# gray = cv2.imread('/content/0_0.jpg', cv2.IMREAD_GRAYSCALE)
# crop_image = crop_image_remove_white(gray)



In [ ]:
def overlay_image_centre(img):
  # grayscale image 
  h, w = img.shape
  # print(h,w)

  square_side = max(h,w)+10

  # load background image as white
  back = np.full((square_side,square_side),255,np.uint8)
  hh, ww = back.shape
  # print(hh,ww)

  # compute xoff and yoff for placement of upper left corner of resized image   
  yoff = round((hh-h)/2)
  xoff = round((ww-w)/2)
  # print(yoff,xoff)

  # use numpy indexing to place the resized image in the center of background image
  result = back.copy()
  result[yoff:yoff+h, xoff:xoff+w] = img

  # view result
  # cv2_imshow(result)
  return result 

# t = overlay_image_centre(crop_image)

# print(type(t))

In [ ]:
# dilation 

def dilation_resize(img):
  img = cv2.resize(img,(100,100)) # resize for inputs to be of same size
  
  # refer - https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html#:~:text=kernel%20%3D%20np.ones((5%2C5)%2Cnp.uint8) 
  # kernel = np.full((5,5),255,np.uint8) # white background 
  kernel = np.ones((5,5),np.uint8) 

  img = (255-img) #invert threshold(b/w) image 

  # erosion = cv2.erode(img,kernel,iterations = 1)

  dilation = cv2.dilate(img,kernel,iterations = 1)

  # opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

  # closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

  img = (255-dilation) #invert threshold(b/w) image 
  img = cv2.resize(img,(28,28))

  return img
  
# img = cv2.imread('/content/0_0.jpg')
# cv2_imshow(dilation_resize(img))

reading dataset files 

In [ ]:
from IPython.core.display import display_markdown
#create training data
training_data = []
CATEGORIES = []
images= []
images_path = []
DATADIR = '/content/dataset'
def count_least_number_images_in_class_training_data():
    count = 10000 # count least images among any of the folder  
    for category in CATEGORIES:  # do 
        index = 0
        path = os.path.join(DATADIR,str(category))  # create path to classes
        class_num = CATEGORIES.index(category)  # get the classification 
        print(path)
        for img in tqdm(os.listdir(path)):  # iterate over each image 
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                if img_array.shape == (180,364) or img_array.shape==(364,180): # added 01-02-2022
                  os.remove(os.path.join(path,img))
                  continue
                index+=1
            except Exception as e:  # in the interest in keeping the output clean...
                pass
        if count>index:
          count=index 
    print('---------------------')
    return count

exception = 0
def create_training_data():
    count = count_least_number_images_in_class_training_data()
    fill = len(str(count))+1 # for giving names to images
    
    for category in CATEGORIES:  # do 
        index = 0
        path = os.path.join(DATADIR,str(category))  # create path to classes
        class_num = CATEGORIES.index(category)  # get the classification  

        for img in tqdm(os.listdir(path)):  # iterate over each image 
            try:
                

                # comment following if statement for unbalanced classes
                if (index+1)>count:
                  #remove that image 
                  os.remove(os.path.join(path,img))
                  continue # for balanced classes
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                # TODO - first cropped image and the preprocess and see result. Current code resizes image at last.
                
                # added 01-02-2022
                img_array = cv2.resize(img_array,(100,100))

                # STEP 1 and 2
                cropped = crop_image_remove_white(img_array)
                
                # STEP 3
                overlay = overlay_image_centre(cropped)
                
                # STEP 4 
                dilation = dilation_resize(overlay)
               
                # for debug 
                # images.append([img_array,cropped,overlay,dilation])
                # ipyplot.plot_images([img_array,cropped,overlay,dilation],['original','cropped_threshold','overlay','dilation'] ,max_images=10, img_width=100)
              

                # Reference - https://stackoverflow.com/questions/18805348/how-rename-the-images-in-folder/47105286#:~:text=import%20os%0Afor%20dirname%20in%20os.listdir(%22.%22)%3A%0A%20%20%20%20if%20os.path.isdir(dirname)%3A%0A%20%20%20%20%20%20%20%20for%20i%2C%20filename%20in%20enumerate(os.listdir(dirname))%3A%0A%20%20%20%20%20%20%20%20%20%20%20%20os.rename(dirname%20%2B%20%22/%22%20%2B%20filename%2C%20dirname%20%2B%20%22/%22%20%2B%20str(i)%20%2B%20%22.bmp%22)
                os.rename(os.path.join(path,img),os.path.join(path,str(category)+"_"+str(index).zfill(fill)+".jpg"))
                cv2.imwrite(os.path.join(path,str(category)+"_"+str(index).zfill(fill)+".jpg"),dilation) # overwrite image 
                index+=1
                
                images_path.append(os.path.join(path,str(category)+"_"+str(index).zfill(fill)+".jpg"))
                # print(path)     
                  
                training_data.append([dilation, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                global exception
                exception+=1
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

            # break # for one iteration break/ debug
        print("\nImages in class",category,"are",index)
        # break # for debug 

print('exceptions occurred :',exception)
create_training_data() #don't use argument aspect_ratio (To use, load function - https://colab.research.google.com/drive/11gfaJEI9Q5D3YhocdW4fe8CK7dbp5uhJ?authuser=1#scrollTo=k1ZLDtYDTtcg&line=1&uniqifier=1 )
print("\n","Length of training data ",len(training_data))
# ipyplot.plot_class_representations(images_path, img_width=150,force_b64=True)


download dataset in zip

In [ ]:
!zip -r /content/preprocessed_dataset.zip /content/dataset

from google.colab import files
files.download("/content/preprocessed_dataset.zip")

shuffle

In [ ]:
import random
random.shuffle(training_data)

# Our training_data is a list, meaning it's mutable, so it's now nicely shuffled. 
#We can confirm this by iterating over a few of the initial samples and printing out the class.
for sample in training_data[:10]:
    print(sample[1])

convert to numpy

In [ ]:
X = [] #features
y = [] #labels

for features,label in training_data:
    X.append(features)
    y.append(label)

print(type(X[0]),X[0].shape)
# print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1)) # Gives error TODO 
# print(np.array(X).shape,np.array(X).reshape(-1, 28, 28, 1).shape,np.array(y).shape)

# X = np.array(X).reshape(-1, 28, 28, 1)   
X=np.array(X)
y = np.array(y)

print(X.shape,y.shape)

plot images class wise

In [ ]:
# ipyplot.plot_class_representations(X,y, img_width=28,force_b64=True)
ipyplot.plot_class_tabs(X, y, max_imgs_per_tab=20, img_width=50)

export to pickle file

In [ ]:
# save data 

import pickle
dataset = ["train","test","validate"]
name = dataset[1] # select name for data files 
pickle_out = open("x_"+name+".pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y_"+name+".pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()
files.download("x_"+name+".pickle")
files.download("y_"+name+".pickle")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>